In [ ]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/NIR/data(markup 1, 2, 3, 5).csv" /content/data.csv

In [ ]:
!cp "/content/drive/MyDrive/NIR/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/NIR/eval.csv" /content/eval.csv

#### Разметка релевантных новостей с активным обучением

In [ ]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [ ]:
class CustomHuggingFaceDatasets(Dataset):
    """
    Support for `huggingface.datasets`: (https://github.com/huggingface/datasets).
    The purpose of this wrapper is to separate the labels from the rest of the sample information
    and make the dataset ready to be used by `baal.active.ActiveLearningDataset`.
    Args:
        dataset (Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter
            sequences.
    """

    def __init__(
        self,
        dataset,
        tokenizer=None,
        target_key: str = "label",
        input_key_1: str = "sentence1",
        input_key_2: str = "sentence1",
        max_seq_len: int = 250,
    ):
        self.dataset = dataset
        self.targets, self.text1, self.text2 = self.dataset[target_key], self.dataset[input_key_1], self.dataset[input_key_2]
        self.targets_list: List = np.unique(self.targets).tolist()
        self.input_ids, self.attention_masks = (
            self._tokenize(tokenizer, max_seq_len) if tokenizer else ([], [])
        )

    @property
    def num_classes(self):
        return len(self.targets_list)

    def _tokenize(self, tokenizer, max_seq_len):
        # For speed purposes, we should use fast tokenizers here, but that is up to the caller
        tokenized = tokenizer(
            self.text1,
            self.text2,
            add_special_tokens=True,
            max_length=max_seq_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return tokenized["input_ids"], tokenized["attention_mask"]

    def label(self, idx: int, value: int):
        """Label the item.
        Args:
            idx: index to label
            value: Value to label the index.
        """
        self.targets[idx] = value

    def __len__(self):
        return len(self.text1)

    def __getitem__(self, idx):
        target = self.targets_list.index(self.targets[idx])

        return {
            "input_ids": self.input_ids[idx].flatten() if len(self.input_ids) > 0 else None,
            "inputs": self.text1[idx] + '\n' + self.text2[idx],
            "attention_mask": self.attention_masks[idx].flatten()
            if len(self.attention_masks) > 0
            else None,
            "label": torch.tensor(target, dtype=torch.long),
        }

def custom_active_huggingface_dataset(
    dataset,
    tokenizer=None,
    target_key: str = "label",
    input_key_1: str = "sentence1",
    input_key_2: str = "sentence2",
    max_seq_len: int = 512,
    **kwargs
):
    """
    Wrapping huggingface.datasets with baal.active.ActiveLearningDataset.
    Args:
        dataset (torch.utils.data.Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter sequences.
        kwargs (Dict): Parameters forwarded to 'ActiveLearningDataset'.
    Returns:
        an baal.active.ActiveLearningDataset object.
    """

    return ActiveLearningDataset(
        CustomHuggingFaceDatasets(dataset, tokenizer, target_key, input_key_1, input_key_2, max_seq_len), **kwargs
    )

Information on the hyperparms below

* epoch: Number of times you want to run and AL loop
* batch_size: The train and eval batch size for hf trainer arguments
* model: Hugging Face Model
* query_size: Number of samples you want to query at each AL iteration for labelling
* heuristic: The acquisition function/heuristic based on which you want to query the important samples
* iterations: The number of iterations you want to run for MCdropout to find the uncertanities
* shuffle_prop: Additional Noise to counter selection bias
* learning_epoch: Traing epochs for hugging face trainer

In [ ]:
hyperparams = {
    "epoch": 6,
    "batch_size": 10,
    "model": "DeepPavlov/rubert-base-cased",
    "query_size": 50,
    "heuristic": "bald",
    "iterations": 10,
    "shuffle_prop": 0.05,
    "learning_epoch": 3,
}

In [ ]:
# Check for CUDA
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

id2label = {0: "LABEL_0", 1: "LABEL_1"}
label2id = {"LABEL_0": 0, "LABEL_1": 1}

# Load Model
hf_model = AutoModelForSequenceClassification.from_pretrained(
    hyperparams["model"], num_labels=2, id2label=id2label, label2id=label2id
    )

# Setup tokenizer for model
tokenizer = AutoTokenizer.from_pretrained(hyperparams["model"])

# Enable dropouts for predictions
hf_model = patch_module(hf_model)

# Send model to device and setup cuda arguments
if use_cuda:
    hf_model.to("cuda:0")
    no_cuda = False
else:
    hf_model.to("cpu")
    no_cuda = True

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
# Define labels in your dataset
label_list = [0, 1]

# Load data from files
data = load_dataset("csv", data_files={'train': '/content/train.csv', 'eval': "/content/eval.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-53afec5e94009a99/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_label_from_data(active_dataset, data, target, input1, input2, indexes) -> List[int]:
    """
    Get labels from the active dataset, this assumes that you have
    already labelled some samples in your initial dataset

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """

    labels = []

    # Now since you labelled points earlier now some part of pool has become train
    # so in order to get the pool indexes based on your 'original' data i.e
    # your raw_train_set. Make sure to user __pool_tp

    raw_data_idx = active_dataset._pool_to_oracle_index(indexes)

    for idx in raw_data_idx:
        #print(f"Adding labels for Raw data Index {idx} : {data[input1][idx]}")
        #print(f"Adding labels for Raw data Index {idx} : {data[input2][idx]}")

        label = data[target][idx]
        labels.append(label)
        #print(label)
        #print("\n")

    return labels

In [ ]:
def get_label_human_oracle(active_dataset, indexes) -> List[int]:
    """
    Get labels from human oracle. During the AL loop some samples
    will go to the human labeller

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """
    # List for corresponding labels
    labels = []

    skipped = []
    
    for sample_idx, idx in enumerate(indexes):

        while True:
            try:
                print(idx)
                pprint(active_dataset.pool.__getitem__(idx)['inputs'], width=150)
                label = int(input())
            except ValueError:
                print("Sorry, I didn't understand that.")
                continue
            if label != -1 and label not in label_list:
                print(f"Allowed labels are {label_list}")
                continue
            if label == -1:
                print("Skipping this sample")
                skipped.append(sample_idx)
                break
            else:
                labels.append(label)
                break
        print("\n")

    indexes_upd = np.delete(indexes, skipped)

    return labels, indexes_upd

In [ ]:
small_data = data['train'].select(range(5_000))
#small_data = data['train']

active_set = custom_active_huggingface_dataset(
    small_data,
    tokenizer=tokenizer,
    target_key='relevant_part',
    input_key_1='content',
    input_key_2="part_body"
)

active_set.can_label = True

# 1000 первых объектов набора данных
label_from_data = get_label_from_data(active_set, small_data, 'relevant_part', 'content', 'part_body', range(1_150))

active_set.label(
    range(1_150),
    label_from_data,
)

valid_set = CustomHuggingFaceDatasets(data['eval'], tokenizer=tokenizer, target_key='relevant_part', input_key_1='content', input_key_2="part_body")

active_set, test_set = active_set, valid_set

In [ ]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

# Model save checkpoint
save_checkpoint = 2

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [ ]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")

    if (epoch != last_epoch):
      # MCdropout to gather uncertanities
      predictions = baal_trainer.predict_on_dataset(
          active_set.pool, iterations=hyperparams["iterations"]
      )
      print("\n")

      # Acquistion of the most informative and diverse samples based on BatchBALD
      top_uncertainty = heuristic(predictions)[: hyperparams.get("query_size", 1)]

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, top_uncertainty
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.326700,0.578277,0.114286
2,0.430300,0.604353,0.433735
3,0.133200,0.580405,0.543478




[666-MainThread  ] [baal.transformers_trainer_wrapper:predict_on_dataset_generator:67] 2023-04-22T15:14:36.853781Z [info     ] Start Predict                  dataset=3850



100%|██████████| 385/385 [23:23<00:00,  3.64s/it]




182
('2 минута. Михнов чуть не пролез с левого края на пятак - сработала подстраховка у хозяев. В ответ Горохов со второй попытки попал в створ с '
 'правого фланга. 3 минута. ПЕТРОВ! Легко обыграл Калимулина, а Вьюхин вдруг выскочил вперед, чем, похоже, смутил Олега! КОторый очень раздосадован. '
 '3 минута. СУББОТИН! 0:1! Делает передачу с левого края в расчете на замыкание Калянина, но после рикошета на пятаке шайба скользнула в ближний '
 'угол! 4 минута. УППЕР! 1:1! Гуськов ошибся с выбором позиции за воротами, Левандовский его опередил и выдал передачу с ближней штанги на дальнюю - '
 'Уппер бросал уже в пустые! 5 минута. Королюк несложно швыряет справа, однако рикошет чуть не сделал бросок голевым. 5 минута. Бросок в упор со '
 'средней дистанции от Булиса парирует Вьюхин! 6 минута. Иргл щелкает справа от борта, шайба попадает в каркас ворот с внешней стороны и покидает '
 'лед.\n'
 'Урычев удален до конца матча за удар коленом. Защитник «Локомотива» Юрий Урычев был удален до 

...


100%|██████████| 365/365 [22:09<00:00,  3.64s/it]




2917
('87 минута. Мовсисян покидает поле. Барриосу дают немного побегать. 88 минута. И Быстров уходит с поля. Вместо него Сердеров. 88 минута. Ой, что '
 'творится-то! Кержаков зачем-то выбежал из штрафной на перехват, хотя мяч контролировал Епуряну. А молдаванин не глядя отдал голкиперу. В итоге, '
 'едва спартаковцы не забили в пустые ворота. 89 минута. Снова "Спартак" на своей половине и отбивается. 90 минута. Боккетти удачно поставил корпус '
 'Максимову и не пустил того к мячу в штрафной. Максимов толкнул итальянца в спину, а тут еще и Митрюшкин в него врезался. В итоге, потолкались - '
 'разошлись. 90 минута. Ромуло меняет Кариоку на последние несколько минут. 90 минута. 5 минут, кстати, добавлено. 90 минута. Прострел Сердерова '
 'неудачный - мяч становится легкой добычей Митрюшкина. 90 минута. гол! Сердеров! 2:2! Вот это да! Фантастика! Бесперспективный заброс Гаджибекова, '
 'и Боккетти зачем-то тронул мяч, тот неудобно прилетает к Ташчи, которого обкрадывает Сердеров и нано

 83%|████████▎ | 5/6 [3:23:17<40:18, 2418.93s/it]  

Epoch,Training Loss,Validation Loss,F1
1,0.451700,0.830417,0.059701
2,0.245000,0.545258,0.354430
3,0.195600,0.713329,0.488372


100%|██████████| 6/6 [3:30:50<00:00, 2108.37s/it]

In [ ]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-360")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.5071448087692261,
 'eval_f1': 0.6666666666666666,
 'eval_runtime': 3.6073,
 'eval_samples_per_second': 73.463,
 'eval_steps_per_second': 7.485}

In [ ]:
!cp -r "/content/checkpoint-390" "/content/drive/MyDrive/NIR/bald-390"

#### Разметка релевантных новостей со случайным обучением

In [ ]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [ ]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")
    if (epoch != last_epoch):
      point_idx_train = np.random.choice(len(active_set.pool) - 1, hyperparams.get("query_size", 1), replace=False)

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, point_idx_train
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # Save model
      #if epoch == save_checkpoint:
      #    save_model(baal_trainer)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.414200,0.331453,0.000000
2,0.382600,0.274154,0.000000
3,0.252500,0.285223,0.000000




7974
('32 минута. "Ростов" неубедителен, прежде всего, в созидании. 32 минута. Калачев получил карточку. 33 минута. Калачев ворвался в штрафную, мощно '
 'пробил в сторону ворот, Шунин сумел парировать в сторону. 35 минута. "Ростов" провел быструю атаку, теперь интересно, продолжат ли динамовцы '
 'активничать или предпочтут идти в атаку меньшими силами?\n'
 'Семшов забил 100-й гол в карьере. Полузащитник «Динамо» Игорь Семшов открыл счет на 11-й минуте гостевого матча 17-го тура премьер-лиги с '
 '«Ростовом». Этот гол стал для футболиста 100-м в карьере. Подробную статистику выступлений Семшова смотрите здесь . ВИДЕО')


  0%|          | 0/4 [06:36<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-390")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.4910638630390167,
 'eval_f1': 0.6666666666666666,
 'eval_runtime': 3.6883,
 'eval_samples_per_second': 71.849,
 'eval_steps_per_second': 7.32}

In [ ]:
!cp -r "/content/checkpoint-61" "/content/drive/MyDrive/NIR/random"